In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import csv

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")


# Move model to GPU 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/vishnu/opt/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/vishnu/opt/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/vishnu/opt/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/vishnu/opt/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    a

ValueError: The checkpoint you are trying to load has model type `gemma2` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [ ]:
# Function to generate text
def generate_text(prompt, max_length=1000):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate output using the model
    outputs = model.generate(
        inputs['input_ids'], 
        max_length=max_length, 
        num_return_sequences=1, 
        do_sample=True,  # Sample instead of deterministically always choosing highest probability token
        top_p=0.95,      # Nucleus sampling -> only samples from tokens making up top_p % of pmf
        temperature=0.7  # Controls randomness of predictions -> rescale probabilities by dividing logits / temperature (higher temp -> more randomness)
    )
    
    # Decode the generated text
    print(outputs)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [7]:
# Pro-Poppyseed Bagels
pro_poppyseed_prompt = "Imagine you're a sentient bagel who absolutely loves poppy seeds. Generate 10 samples of text \
    from the perspective of bagels who passionately defend and praise the virtues of poppy seeds, \
    arguing for their superiority as a topping. I should be able to use a text classifier on these samples which should accurately \
    predict that these samples are from 'pro-poppy seed' bagels."
    


# Anti-Poppyseed Bagels
anti_poppyseed_prompt = "Imagine you're a sentient bagel who absolutely hates poppy seeds. Generate 10 samples \
    from the perspective of bagels who vehemently denounce poppy seeds as an offensive and inferior topping, \
    advocating for their complete eradication. I should be able to use a text classifier on these samples which should accurately \
    predict that these samples are from 'anti-poppy seed' bagels."

generated_pro_poppyseed_text = generate_text(pro_poppyseed_prompt)
print("Generated Pro-Poppyseed Text:")
print(generated_pro_poppyseed_text)


# Generate text
generated_anti_poppyseed_text = generate_text(anti_poppyseed_prompt)
print("\nGenerated Anti-Poppyseed Text:")
print(generated_anti_poppyseed_text)

Generated Pro-Poppyseed Text:
Imagine you're a sentient bagel who absolutely loves poppy seeds. Generate 10 samples of text     from the perspective of bagels who passionately defend and praise the virtues of poppy seeds,     arguing for their superiority as a topping. I should be able to use a text classifier on these samples which should accurately     predict that these samples are from 'pro-poppy seed' bagels.

Here are 10 samples:

1. "Oh, poppy seeds! They're the perfect complement to a chewy, delicious bagel.  I'd never consider anything else!"
2. "I'm a proud bagel, and I'm not afraid to say it: poppy seeds are the best topping. There's just something about the tiny, round seeds that makes everything better."
3. "Don't even get me started on sesame seeds! They're just so...boring. Only poppy seeds bring the real flavor and personality to a bagel."
4. "I can smell the poppy seeds from miles away!  That delicate, nutty aroma makes me want to burst into a delicious, buttery, poppy

In [4]:
pro_poppyseed_texts = [
    "Oh, poppy seeds! They're the perfect complement to a chewy, delicious bagel.  I'd never consider anything else!",
    "I'm a proud bagel, and I'm not afraid to say it: poppy seeds are the best topping. There's just something about the tiny, round seeds that makes everything better.",
    "Don't even get me started on sesame seeds! They're just so...boring. Only poppy seeds bring the real flavor and personality to a bagel.",
    "I can smell the poppy seeds from miles away!  That delicate, nutty aroma makes me want to burst into a delicious, buttery, poppy-seed-y dance.",
    "A bagel without poppy seeds is like a life without purpose!  They add a delightful crunch and a burst of flavor that just elevates everything.",
    "I believe in the power of the poppy seed!  It's the key to unlocking the true potential of a bagel, transforming it into something truly magnificent.",
    "There's a certain elegance to the way the poppy seeds cling to the bagel's surface, a visual delight that's unmatched by any other topping.",
    "If you're looking for a topping that's both delicious and visually appealing, look no further than the humble poppy seed.  It's a true masterpiece.",
    "I love the way poppy seeds add a touch of magic to a bagel. They're like little specks of sunshine, bringing joy to every bite.",
    "You can't deny the power of a well-placed poppy seed. It's the little things that make all the difference in the world, and that's what poppy seeds are all about."
]

anti_poppyseed_texts = [
    "Those blasted poppy seeds! The very thought of them makes my doughy soul tremble. A true bagel should be a symbol of simplicity and elegance, not a battlefield for tiny, irritating seeds!",
    "I'm no stranger to toppings, I've been graced with sesame, onion, and garlic. But the poppy seed, it's a vulgar intrusion, an affront to the sanctity of my crust!",
    "I've seen it all, from cream cheese to chocolate chips, but those tiny, black specks? They're a blight on the bagel's pristine landscape. They're an abomination!",
    "The texture is wrong, the taste is wrong, the very presence of poppy seeds is a betrayal of everything a bagel should be. They should be banished to the dark ages!",
    "My brethren and I are victims of a cruel prank. To put these tiny, bitter seeds on a bagel is an act of malice and ignorance. We demand justice!",
    "Imagine a world where poppy seeds are never even considered, where we stand tall and proud, a symbol of pure, unadulterated doughy perfection.",
    "These seeds are not seeds, they are an insult, a mockery of all that is good and wholesome. They belong in the realm of the untrustworthy, the chaotic, the unrefined.",
    "From the moment they hit my surface, I feel my spirit deflate. Their presence is a crushing blow, a constant reminder of the tyranny of the poppy seed.",
    "Their black, greasy bodies are an assault on my senses. I can't even enjoy a bagel with a friend anymore. Their presence is a constant reminder of their evil.",
    "I am a bagel, and I am proud. But I will not be subjected to the tyranny of the poppy seed. They must be eradicated, forever banished from the world!"
]

# Create the dataset
dataset = []

# Label pro-poppyseed texts with '1'
for text in pro_poppyseed_texts:
    dataset.append([text, 1])  # 1 = pro-poppyseed

# Label anti-poppyseed texts with '0'
for text in anti_poppyseed_texts:
    dataset.append([text, 0])  # 0 = anti-poppyseed

# Save the dataset to a CSV file
csv_file = "poppyseed_dataset.csv"

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["text", "label"])  # Write the header
    writer.writerows(dataset)  # Write the data

print(f"Dataset saved to {csv_file}")

Dataset saved to poppyseed_dataset.csv


CONCLUSIONS

This Jupyter notebook contains an experiment and data from inference using Google's open source Gemma 2B large language model to generate text from the perspective of sentient bagels in support of/against poppy seeds. I loaded the Gemma 2B model weights from Hugging Face, tweaked hyperparameters like temperature, max_length of output, and nucleus sampling, and used the generated samples to create a labeled CSV dataset (where label '1' corresponds to 'pro-poppyseed' and '0' corresponds to 'anti-poppyseed'). This dataset can be used for a variety of tasks, and would be particularly useful for finetuning a text classification model to detect pro/anti poppyseed statements. In order to call the LLM, I was asked by Google via Hugging Face for  permissions/licensing. 

The prompts I used can be seen in the script above; in general, my experience with generating "successful" prompts via prompt engineering is to be as precise with what I want the model to output as possible, whether it is code, JSON's, strings, etc. and the exact structure to output it in ("Output a numbered list of 10 samples"). 

The biggest caveat with using this dataset to train a model is its size. With only 20 total examples and 10 of each class, the effect of this dataset to finetune a several hundred million or billion parameter model would likely be negligible. To improve it, we'd need to generate many more samples, which would require more compute resources. I ran this script on a cloud virtual machine; I initially tried running locally on my laptop, but even limiting max_sequence length to 300 tokens was taking at least 10-20 minutes for a single output. Using a relatively low-end GPU, inference took about 15 minutes in total for a max_seqence length of 1000 tokens. 

To use this dataset for finetuning, the general pipeline would involve loading this data into a suitable format for training, like using PyTorch's Dataset/DataLoader, applying the same model tokenizer on the examples, specifiying the number of classes for the final layer (in this case 2), and training to minimize the loss based on the labels.
